# SKRUD Prototype API
> A SKRUD API must implement a few headers, and be disciplined about its HTTP. This demo starts one inside your current app

In [1]:
from uuid import uuid4
import ipywidgets as W, traitlets as T
from tornado import web, escape, httpclient
from notebook.utils import url_path_join as ujoin
from pyld import jsonld
import pandas as pd

In [2]:
with __import__("importnb").Notebook():
    from WebApp import make_notebook_app
    from ScrudWidgetHandler import ScrudWidgetHandler
    from ScrudTableHandler import ScrudTableHandler
    from CPUTemperature import CPUTemperatureSlider
    import ScrudTemperatureTable, ScrudTemperature
    from ScrudAPI import ScrudAPI

In [3]:
app = make_notebook_app()

[I 13:07:32.208 NotebookApp] The port 8877 is already in use, trying another port.
[I 13:07:32.261 NotebookApp] JupyterLab extension loaded from /home/weg/Documents/work/qs/skrud/envs/default/lib/python3.7/site-packages/jupyterlab
[I 13:07:32.262 NotebookApp] JupyterLab application directory is /home/weg/Documents/work/qs/skrud/envs/default/share/jupyter/lab


Install the handlers

In [4]:
full_url = app.display_url.split("?")[0]

In [5]:
scrud = ScrudAPI(full_url=full_url, 
                 base_url=app.base_url,
                 tornado_app=app.web_app,)

In [6]:
scrud.add_handler(ScrudTemperature.default_handlers)
scrud.add_handler(ScrudTemperatureTable.default_handlers)

In [7]:
client = httpclient.AsyncHTTPClient()

async def fetch(*bits, **fetch_args):
    url = ujoin(scrud.api_url, *bits) + f"?token={app.token}"
    print(url)
    r = await client.fetch(url, **fetch_args)
    return escape.json_decode(r.body or "null"), dict(r.headers)